# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
pokemon = pd.read_csv("pokemon.csv")
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [3]:
pokemon['Name'] = pokemon['Name'].apply(lambda x: 'Mega ' + x.split('Mega')[-1] if 'Mega' in x else x)


# test transformed data
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,Mega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,Mega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [4]:
# Hypothesis: The 'Total' column is the sum of 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', and 'Speed'.

# Sum of the specified columns
pokemon['Computed Total'] = pokemon[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].sum(axis=1)

# Check if computed total equals the 'Total' column for all rows
all_totals_match = (pokemon['Total'] == pokemon['Computed Total']).all()

# Drop the 'Computed Total'
pokemon.drop('Computed Total', axis=1, inplace=True)

if (all_totals_match):
    print("The 'Total' column is the sum of 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', and 'Speed'")
else:
    print("The 'Total' column is NOT the sum of 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', and 'Speed'")

The 'Total' column is the sum of 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', and 'Speed'


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [5]:
unique_type1 = set(pokemon['Type 1'].dropna().unique())
unique_type2 = set(pokemon['Type 2'].dropna().unique())

distinct_types_merged = unique_type1.union(unique_type2)
distinct_types_merged

# Initialize a new DataFrame with zeros and the same number of rows as pokemon
type_encoded_df = pd.DataFrame(0, index=pokemon.index, columns=list(distinct_types_merged))

# Loop through each Pokémon
for idx, row in pokemon.iterrows():
    if row['Type 1'] in distinct_types_merged:
        type_encoded_df.at[idx, row['Type 1']] = 1
    if row['Type 2'] in distinct_types_merged:
        type_encoded_df.at[idx, row['Type 2']] = 1

type_encoded_df.head()

,Normal,Flying,Fairy,Electric,Psychic,Dark,Fighting,Ghost,Water,Ice,Dragon,Bug,Ground,Rock,Grass,Steel,Poison,Fire
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [6]:
# Concatenate the original 'Total' column with the encoded dataframe
combined_df = pd.concat([pokemon['Total'], type_encoded_df], axis=1)

# Compute the correlation with 'Total'
correlation = combined_df.corr()['Total']

# Drop the 'Total' correlation with itself
correlation = correlation.drop('Total')

# Sort the correlations in descending order
sorted_correlation = correlation.sort_values(ascending=False)

sorted_correlation

Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
Name: Total, dtype: float64

The Pokemon type Dragon has the highest positive correlation with the Total value, indicating that Pokemon with this type tend to have higher overall stats.
The Pokemon type Psychic is second.
Bug Pokemon have a negative correlation with the Total value, indicating they may have lower overall stats on average.
First impression: we should opt for a Pokemon of Psychic Dragon.

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [7]:
pokemon.sort_values(by = 'Total', ascending=False).head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
426,384,Mega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True
164,150,Mega Mewtwo Y,Psychic,NaN,780,106,150,70,194,120,140,1,True
163,150,Mega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
422,382,KyogrePrimal Kyogre,Water,NaN,770,100,150,90,180,160,90,3,True
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True
552,493,Arceus,Normal,NaN,720,120,120,120,120,120,120,4,True
712,646,KyuremWhite Kyurem,Dragon,Ice,700,125,120,90,170,100,95,5,True
711,646,KyuremBlack Kyurem,Dragon,Ice,700,125,170,100,120,90,95,5,True
409,373,Mega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False
413,376,Mega Metagross,Steel,Psychic,700,80,145,150,105,110,110,3,False


Hypothesis 1: We choose the types of the Pokemon that has the highest Total score.

In [8]:
hypothesis_combination_dr_fl_ = (
    pokemon[
        ((pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Flying')) | 
        ((pokemon['Type 2'] == 'Dragon') & (pokemon['Type 1'] == 'Flying')) 
    ]
)
hypothesis_combination_dr_fl_

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
161,149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,1,False
365,334,Altaria,Dragon,Flying,490,75,70,90,70,105,80,3,False
408,373,Salamence,Dragon,Flying,600,95,135,80,110,80,100,3,False
409,373,Mega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False
425,384,Rayquaza,Dragon,Flying,680,105,150,90,150,90,95,3,True
426,384,Mega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False
791,715,Noivern,Flying,Dragon,535,85,70,80,97,80,123,6,False


In [9]:
hypothesis_combination_dr_fl_['Total'].mean()

578.75

In [10]:
hypothesis_combination_dr_fl = (
    pokemon[
        ((pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Flying')) 
    ]
)
hypothesis_combination_dr_fl

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
161,149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,1,False
365,334,Altaria,Dragon,Flying,490,75,70,90,70,105,80,3,False
408,373,Salamence,Dragon,Flying,600,95,135,80,110,80,100,3,False
409,373,Mega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False
425,384,Rayquaza,Dragon,Flying,680,105,150,90,150,90,95,3,True
426,384,Mega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True


In [11]:
hypothesis_combination_dr_fl['Total'].mean()

641.6666666666666

Conflusion: Even though we might be lucky and get Mega Rayquaza with 780 Total score, we might also be unlucky and get Noibat with 245 Total score.

In [12]:
Hypothesis 2: We choose the types that are correlated most with the Total. In this case 

SyntaxError: invalid syntax (4151607828.py, line 1)

In [ ]:
hypothesis_combination_dr_ps = (
    pokemon[
        ((pokemon['Type 1'] == 'Dragon') & (pokemon['Type 2'] == 'Psychic')) | 
        ((pokemon['Type 2'] == 'Dragon') & (pokemon['Type 1'] == 'Psychic')) 
    ]
)
hypothesis_combination_dr_ps

In [ ]:
hypothesis_combination_dr_ps['Total'].mean()

'Dragon' 'Psychic' is a better choice compared to 'Dragon' 'Flying'. Let's see if we can find a better option. 

In [ ]:
type_combinations = pokemon.groupby(['Type 1', 'Type 2']).mean(['Total'])

top_combination = type_combinations.sort_values('Total', ascending=False).head(10)
top_combination

Neighter of the options are in the top 10. The winner is 'Ground' 'Fire' with mean Total = 770.

In [ ]:
hypothesis_combination_gr_fi = (
    pokemon[
        ((pokemon['Type 1'] == 'Ground') & (pokemon['Type 2'] == 'Fire')) 
    ]
)
hypothesis_combination_gr_fi

p.s. I still have second thoughts what to choose :)